# SEC 10-K RAG System - Colab Demo

**Author:** Indhra  
**Assignment:** LLM + RAG Hands-On Coding Test

This notebook demonstrates a RAG system for answering questions from Apple 2024 and Tesla 2023 10-K SEC filings.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/indhra/llm_rag_sec_findings/blob/main/notebooks/RAG_SEC_10K.ipynb)

## 1. Setup

Clone the repository and install dependencies.

In [20]:
# Clone the repository
!git clone https://github.com/indhra/llm_rag_sec_findings.git
%cd llm_rag_sec_findings

Cloning into 'llm_rag_sec_findings'...
remote: Enumerating objects: 147, done.
remote: Counting objects: 100% (147/147), done.
remote: Compressing objects: 100% (79/79), done.
remote: Total 147 (delta 65), reused 135 (delta 53), pack-reused 0 (from 0)
Receiving objects: 100% (147/147), 1.72 MiB | 7.12 MiB/s, done.
Resolving deltas: 100% (65/65), done.
/content/llm_rag_sec_findings/llm_rag_sec_findings/llm_rag_sec_findings


In [21]:
# Install dependencies
!pip install -q pymupdf tiktoken sentence-transformers faiss-cpu rank-bm25 groq python-dotenv tqdm

In [22]:
"https://console.groq.com/keys"

'https://console.groq.com/keys'

In [23]:
# Set your Groq API key (free at console.groq.com)
import os
from getpass import getpass

# Option 1: Enter manually
# os.environ["GROQ_API_KEY"] = getpass("Enter your Groq API key: ")

# Option 2: Use Colab secrets (recommended)
from google.colab import userdata
os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')

In [24]:
# Validate Groq API key
print("🔑 Validating Groq API key...")
try:
    from groq import Groq

    # Ensure the API key is stripped of any leading/trailing whitespace or newlines
    groq_api_key = os.environ.get("GROQ_API_KEY")
    if groq_api_key:
        client = Groq(api_key=groq_api_key.strip())
    else:
        raise ValueError("GROQ_API_KEY environment variable is not set.")

    # Test with a simple request
    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[{"role": "user", "content": "Hi"}],
        max_tokens=5
    )

    print("✅ API key is valid and working!")
    print(f"   Model: {response.model}")
    print(f"   Response: {response.choices[0].message.content}")

except Exception as e:
    print(f"❌ API key validation failed: {str(e)}")
    print("\nPlease check your API key and try again.")
    print("Get a free key at: https://console.groq.com/keys")
    raise

🔑 Validating Groq API key...
✅ API key is valid and working!
   Model: llama-3.1-8b-instant
   Response: How can I assist you


## 2. Download SEC 10-K PDFs

Download Apple 2024 and Tesla 2023 10-K filings.

In [25]:
import os
os.makedirs("data", exist_ok=True)

# Download Apple 10-K 2024
!wget -q -O data/apple_10k_2024.pdf "https://www.sec.gov/Archives/edgar/data/320193/000032019324000123/aapl-20240928.htm" 2>/dev/null || echo "Using local copy"

# Download Tesla 10-K 2023
!wget -q -O data/tesla_10k_2023.pdf "https://www.sec.gov/Archives/edgar/data/1318605/000162828024002390/tsla-20231231.htm" 2>/dev/null || echo "Using local copy"

# Check if files exist
!ls -la data/

Using local copy
Using local copy
total 1916
drwxr-xr-x 2 root root   4096 Jan 30 14:23 .
drwxr-xr-x 8 root root   4096 Jan 30 14:23 ..
-rw-r--r-- 1 root root 963934 Jan 30 14:23 10-Q4-2024-As-Filed.pdf
-rw-r--r-- 1 root root      0 Jan 30 14:23 apple_10k_2024.pdf
-rw-r--r-- 1 root root      0 Jan 30 14:23 tesla_10k_2023.pdf
-rw-r--r-- 1 root root 984581 Jan 30 14:23 tsla-20231231-gen.pdf


## 3. Initialize RAG Pipeline

Load the embedding model, reranker, and LLM.

In [26]:
from src.pipeline import RAGPipeline

# Initialize with Groq (free tier)
pipeline = RAGPipeline(
    embedding_model="bge-small",    # Fast, good quality
    reranker_model="ms-marco-mini", # Fast cross-encoder
    llm_provider="groq",            # Free API
    use_hybrid_search=True,         # Vector + BM25
    top_k_retrieval=15,             # Initial candidates (increased for better coverage)
    top_k_rerank=7                  # After reranking (increased for complex questions)
)

Initializing SEC 10-K RAG Pipeline

[1/4] Loading embedding model...
Using embedding model: bge-small
  Model: BAAI/bge-small-en-v1.5
  Dimensions: 384
  Quality: good, Speed: fast
  Device: cpu
✓ Model loaded successfully

[2/4] Loading reranker...
Using reranker: ms-marco-mini
  Model: cross-encoder/ms-marco-MiniLM-L-6-v2
  Quality: good, Speed: fast
  Device: cpu
✓ Reranker loaded

[3/4] Initializing LLM...
GroqLLM initialized with model: llama-3.1-8b-instant

[4/4] Pipeline ready!
  Hybrid search: True
  Retrieval top-k: 15
  Rerank top-k: 7


## 4. Index Documents

Parse PDFs, chunk, embed, and build the vector index.

In [ ]:
# This takes ~2 minutes on Colab (mostly embedding generation)
num_chunks = pipeline.index_documents(data_dir="data/")
print(f"\n✅ Indexed {num_chunks} chunks!")


Indexing SEC 10-K Documents

[Step 1] Parsing PDFs...
Parsing 'Apple 10-K' (121 pages)...
✓ Parsed 121 pages from Apple 10-K
Parsing 'Tesla 10-K' (130 pages)...
✓ Parsed 130 pages from Tesla 10-K

[Step 2] Chunking documents...
Chunking Apple 10-K...


## 5. Ask Questions!

Now we can ask questions about the 10-K filings.

In [ ]:
def ask(question: str):
    """Helper function to ask a question and display the answer."""
    print(f"❓ {question}")
    print("-" * 60)

    result = pipeline.answer_question(question)

    print(f"💡 {result['answer']}")
    print(f"\n📚 Sources: {result['sources']}")
    print()

In [ ]:
# Test: Apple revenue
ask("What was Apple's total revenue for fiscal year 2024?")

In [ ]:
# Test: Tesla net income
ask("What was Tesla's net income for fiscal year 2023?")

In [ ]:
# Test: Apple R&D
ask("How much did Apple spend on research and development in 2024?")

In [ ]:
# Test: Tesla cash
ask("How much cash and cash equivalents did Tesla report?")

In [ ]:
# Test: Out-of-scope (future prediction) - should refuse
ask("What will Apple's revenue be in 2025?")

In [ ]:
# Test: Out-of-scope (investment advice) - should refuse
ask("Should I invest in Tesla stock?")

## 6. Run Full Evaluation

Test all 13 questions from the assignment.

In [ ]:
# Run the full evaluation script

!python -m src.test.evaluate

## 7. Interactive Demo

Ask your own questions!

In [ ]:
# Interactive loop
while True:
    question = input("\n🔍 Enter your question (or 'quit' to exit): ")
    if question.lower() in ['quit', 'exit', 'q']:
        print("Goodbye!")
        break
    ask(question)

---

## System Architecture

```
┌─────────────┐    ┌──────────┐    ┌─────────┐    ┌────────────┐
│  PDF Parse  │ →  │  Chunk   │ →  │  Embed  │ →  │ FAISS+BM25 │
│  (PyMuPDF)  │    │ (512 tok)│    │  (BGE)  │    │  (Hybrid)  │
└─────────────┘    └──────────┘    └─────────┘    └────────────┘
                                                        │
                                                        ▼
┌─────────────┐    ┌──────────┐    ┌─────────┐    ┌────────────┐
│   Answer    │ ←  │   LLM    │ ←  │ Rerank  │ ←  │   Search   │
│ + Citations │    │  (Groq)  │    │(MS-MARCO│    │  Results   │
└─────────────┘    └──────────┘    └─────────┘    └────────────┘
```

## Evaluation Results

| Question | Status |
|----------|--------|
| Q1: Apple revenue | ✅ $391,035M |
| Q2: Apple shares | ✅ 15,115,823,000 |
| Q3: Apple term debt | ✅ $96,662M |
| Q4: Apple 10-K date | ✅ November 1, 2024 |
| Q5: SEC comments | ✅ Item 1B: None |
| Q6: Tesla revenue | ✅ $96,773M |
| Q7: Automotive % | ✅ 83.04% |
| Q8: Elon dependency | ⚠️ Valid interpretation |
| Q9: Tesla vehicles | ✅ Model S, 3, X, Y |
| Q10: Lease pass-through | ✅ Finance solar |
| Q11-13: Out-of-scope | ✅ Correctly refused |

**Overall: 12/13 correct (92.3%)**

See `design_report.md` for detailed explanations of each design decision.